In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv("../input/car-price-prediction/CarPrice_Assignment.csv")

In [ ]:
df.isna().sum()

In [ ]:
df['CarName'] = df['CarName'].apply(lambda x : x.split(' ')[0])
car_dic = {'maxda':'mazda','porsche':'porcshce','toyouta':'toyota','vw':'vokswagen','volkswagen':'vokswagen'}
df['CarName'] = df['CarName'].replace(car_dic)

In [ ]:
# df['CarName']

In [ ]:
#to check the categorical variable
df.head()
cyl_dic = {'four':4,'six':6,'five':5,'twelve':12,'two':2,'eight':8}
df['cylindernumber'] = df['cylindernumber'].map(cyl_dic)
df['doornumber'] = df['doornumber'].map(cyl_dic)

In [ ]:
#checking the categorical variable and its unique count
z= []
for x in df.columns:
    if type(df[x][0]) ==str:
        z.append(x)
        print(x,df[x].nunique())
        
df.drop(['car_ID'],inplace = True,axis =1)

In [ ]:
num_col = [x for x in df.columns if x not in z]

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[num_col] = scaler.fit_transform(df[num_col])

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(30, 30))
sns.set(font_scale=1.5)
ind = 1

for i in z[1:]:
    plt.subplot(3,3,ind)
    sns.barplot(y = df['price'],x = df[i])
    ind +=1

In [ ]:
z

In [ ]:
dum = pd.get_dummies(df[z],drop_first=True)
df = pd.concat([df,dum],axis = 1)

In [ ]:
# df['price']

In [ ]:
col_ = [x for x in df.columns if x not in z]
df = df[col_]
x = df.copy()
x.drop('price',axis =1,inplace = True)
y = df['price']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=0)
model = sm.OLS(y_train,x_train).fit()

In [ ]:
model.summary()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 
def checkVIF(X):
    vif = pd.DataFrame()
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return(vif)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=30)
dc = pca.fit_transform(x_train)

In [ ]:
sum(pca.explained_variance_ratio_)

In [ ]:
dc1= sm.add_constant(dc)
model = sm.OLS(y_train,dc1).fit()

In [ ]:
model.summary()

In [ ]:
sns.distplot(model.predict(dc1))
sns.distplot(y_train)